In [4]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())


                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [5]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [6]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [7]:
sequences = tokenizer.texts_to_sequences(texts)

In [8]:
padded_sequences

array([[   0,    0,    0, ...,    3,   21, 1254],
       [   0,    0,    0, ...,    2,  494,  437],
       [   0,    0,    0, ...,   29,    5, 3496],
       ...,
       [   0,    0,    0, ...,    3,  101, 1331],
       [   0,    0,    0, ...,  339,    8,   42],
       [   0,    0,    0, ...,   25, 3585,   12]])

In [9]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [10]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [11]:
one_hot_labels

array([[0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.]], dtype=float32)

In [12]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [13]:
one_hot_labels[0]

array([0., 0., 0., 0., 1., 0.], dtype=float32)

In [14]:

# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))



Epoch 1/10


400/400 [==============================] - 12s 28ms/step - loss: 1.3751 - accuracy: 0.4746 - val_loss: 0.8757 - val_accuracy: 0.7144
Epoch 2/10
400/400 [==============================] - 11s 27ms/step - loss: 0.3678 - accuracy: 0.8890 - val_loss: 0.5366 - val_accuracy: 0.8181
Epoch 3/10
400/400 [==============================] - 11s 27ms/step - loss: 0.0655 - accuracy: 0.9820 - val_loss: 0.5013 - val_accuracy: 0.8375
Epoch 4/10
400/400 [==============================] - 11s 28ms/step - loss: 0.0274 - accuracy: 0.9943 - val_loss: 0.5257 - val_accuracy: 0.8388
Epoch 5/10
400/400 [==============================] - 11s 28ms/step - loss: 0.0184 - accuracy: 0.9960 - val_loss: 0.5540 - val_accuracy: 0.8344
Epoch 6/10
400/400 [==============================] - 12s 29ms/step - loss: 0.0160 - accuracy: 0.9969 - val_loss: 0.5880 - val_accuracy: 0.8366
Epoch 7/10
400/400 [==============================] - 14s 34ms/step - loss: 0.0144 - accuracy: 0.9970 - val_loss: 0.6106 - val_accur

In [22]:
input_text = "i am ever feeling nostalgic about the fireplace i will know that it is still on the property"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label[0])

1/1 [==============================] - 0s 22ms/step
love
